In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import cv2


In [7]:
train = pd.read_csv('./data/written_name_train_v2.csv')
valid = pd.read_csv('./data/written_name_validation_v2.csv')

In [8]:
train.head

&lt;bound method NDFrame.head of                 FILENAME       IDENTITY
0        TRAIN_00001.jpg      BALTHAZAR
1        TRAIN_00002.jpg          SIMON
2        TRAIN_00003.jpg          BENES
3        TRAIN_00004.jpg        LA LOVE
4        TRAIN_00005.jpg         DAPHNE
...                  ...            ...
330956  TRAIN_330957.jpg          LENNY
330957  TRAIN_330958.jpg        TIFFANY
330958  TRAIN_330959.jpg  COUTINHO DESA
330959  TRAIN_330960.jpg         MOURAD
330960  TRAIN_330961.jpg        HELOISE

[330961 rows x 2 columns]&gt;

In [9]:
#Cleaning the data 
print("Number of train NaN:",train['IDENTITY'].isnull().sum())
print("Number of valid NaN:",valid['IDENTITY'].isnull().sum())

Number of train NaN: 565
Number of valid NaN: 78


In [10]:
#Dropping NaN Values 
train.dropna(axis=0,inplace=True)
valid.dropna(axis=0,inplace=True)

In [11]:
#Checking for unreadable images 
unreadable = train[train['IDENTITY'] == 'UNREADABLE']
unreadable.reset_index(inplace = True, drop=True)
train = train[train['IDENTITY'] != 'UNREADABLE']
valid = valid[valid['IDENTITY'] != 'UNREADABLE']

In [12]:
#Converting the lowercase to uppercase to maintain uniformity
train['IDENTITY']=train['IDENTITY'].str.upper()
valid['IDENTITY']=valid['IDENTITY'].str.upper()

In [13]:
#Reseting the index of elements
train.reset_index(inplace=True,drop=True)
valid.reset_index(inplace=True,drop=True)

In [14]:
#PREPROCESSING AND PREPARING THE IMAGES

#Image loaded as grayscale and reshaped to 256 and 64
#The width and height are cropped if they are greater than 256 and 64 respectively. If they are smaller, then the image is padded with white pixels. Finally the image is rotated clockwise to bring the image shape to (x, y).


#The image is then normalized to range [0, 1]

def preprocess(img):
    (h,w)=img.shape
    final_img = np.ones([64,256])*255 

    if w>256:
        img=img[:,:256]
    
    if h>64:
        img = img[:64,:]

    final_img[:h,:w] = img
    return cv2.rotate(final_img,cv2.ROTATE_90_CLOCKWISE)

In [15]:
#Max size is 30000 train size 
#Max valid size is 3000 
train_size = 1000
valid_size=50

In [16]:
train_x=[]
for i in range(train_size):
    img_dir = './data/train_v2/train/'+train.loc[i,'FILENAME']
    image = cv2.imread(img_dir,cv2.IMREAD_GRAYSCALE)
    image = preprocess(image)
    image = image/255
    train_x.append(image)

In [17]:
valid_x = []
for i in range(valid_size):
        img_dir = './data/validation_v2/validation/'+valid.loc[i,'FILENAME']
        image = cv2.imread(img_dir,cv2.IMREAD_GRAYSCALE)
        image = preprocess(image)
        image = image/255
        valid_x.append(image)

In [18]:
#Reshaping the array
train_x = np.array(train_x).reshape(-1, 256, 64, 1)
valid_x = np.array(valid_x).reshape(-1, 256, 64, 1)


In [19]:
alphabets = u"ABCDEFGHIJKLMNOPQRSTUVWXYZ-'"
max_str_len = 24 #max length of the input labels
num_of_characters = len(alphabets)+1 #+1 for blank
num_of_timestamps = 64 #max length of the predicted labels


In [20]:
def label_to_num(label):
    label_num=[]
    for ch in label:
        label_num.append(alphabets.find(ch))
    return np.array(label_num)

def num_to_label(num):
    ret = ""
    for ch in num:
        if ch == -1:
            break
        else:
            ret+=alphabets[ch]
    return ret


In [21]:
name = 'VISHAL'
print(name,'\n',label_to_num(name))

VISHAL 
 [21  8 18  7  0 11]


In [22]:
train_y = np.ones([train_size,max_str_len])*-1
train_label_len = np.zeros([train_size,1])
train_input_len = np.ones([train_size,1])*(num_of_timestamps-2)
train_output = np.zeros([train_size])

for i in range(train_size):
    train_label_len[i] = len(train.loc[i,'IDENTITY'])
    train_y[i,0:len(train.loc[i,'IDENTITY'])] = label_to_num(train.loc[i,'IDENTITY'])


In [23]:
valid_y = np.ones([valid_size, max_str_len]) * -1
valid_label_len = np.zeros([valid_size, 1])
valid_input_len = np.ones([valid_size, 1]) * (num_of_timestamps-2)
valid_output = np.zeros([valid_size])

for i in range(valid_size):
    valid_label_len[i] = len(valid.loc[i, 'IDENTITY'])
    valid_y[i, 0:len(valid.loc[i, 'IDENTITY'])]= label_to_num(valid.loc[i, 'IDENTITY'])    

In [24]:
print('True label : ',train.loc[100, 'IDENTITY'] , '\ntrain_y : ',train_y[100],'\ntrain_label_len : ',train_label_len[100], 
      '\ntrain_input_len : ', train_input_len[100])

True label :  NOUR 
train_y :  [13. 14. 20. 17. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1.] 
train_label_len :  [4.] 
train_input_len :  [62.]


In [27]:
from keras.models import Model
from keras.layers import Input,Conv2D,MaxPooling2D,Reshape,Bidirectional,LSTM,Dense,Lambda,Activation,BatchNormalization,Dropout
from keras.optimizers import Adam

In [30]:
input_data = Input(shape=(256,64,1),name='input')

inner = Conv2D(32, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal')(input_data)  
inner = BatchNormalization()(inner)
inner = Activation('relu')(inner)
inner = MaxPooling2D(pool_size=(2, 2), name='max1')(inner)

inner = Conv2D(64, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal')(inner)
inner = BatchNormalization()(inner)
inner = Activation('relu')(inner)
inner = MaxPooling2D(pool_size=(2, 2), name='max2')(inner)
inner = Dropout(0.3)(inner)

inner = Conv2D(128, (3, 3), padding='same', name='conv3', kernel_initializer='he_normal')(inner)
inner = BatchNormalization()(inner)
inner = Activation('relu')(inner)
inner = MaxPooling2D(pool_size=(1, 2), name='max3')(inner)
inner = Dropout(0.3)(inner)

# CNN to RNN
inner = Reshape(target_shape=((64, 1024)), name='reshape')(inner)
inner = Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1')(inner)

## RNN
inner = Bidirectional(LSTM(256, return_sequences=True), name = 'lstm1')(inner)
inner = Bidirectional(LSTM(256, return_sequences=True), name = 'lstm2')(inner)

## OUTPUT
inner = Dense(num_of_characters, kernel_initializer='he_normal',name='dense2')(inner)
y_pred = Activation('softmax', name='softmax')(inner)

model = Model(inputs=input_data, outputs=y_pred)
model.summary()


TypeError: object of type &#39;Dense&#39; has no len()

In [ ]:
# the ctc loss function
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [ ]:
labels = Input(name='gtruth_labels', shape=[max_str_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

ctc_loss = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])
model_final = Model(inputs=[input_data, labels, input_length, label_length], outputs=ctc_loss)

In [ ]:
# the loss calculation occurs elsewhere, so we use a dummy lambda function for the loss
model_final.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=Adam(lr = 0.0001))

model_final.fit(x=[train_x, train_y, train_input_len, train_label_len], y=train_output, 
                validation_data=([valid_x, valid_y, valid_input_len, valid_label_len], valid_output),
                epochs=60, batch_size=128)

In [ ]:
preds = model.predict(valid_x)
decoded = K.get_value(K.ctc_decode(preds, input_length=np.ones(preds.shape[0])*preds.shape[1], 
                                   greedy=True)[0][0])

prediction = []
for i in range(valid_size):
    prediction.append(num_to_label(decoded[i]))

In [ ]:
y_true = valid.loc[0:valid_size, 'IDENTITY']
correct_char = 0
total_char = 0
correct = 0

for i in range(valid_size):
    pr = prediction[i]
    tr = y_true[i]
    total_char += len(tr)
    
    for j in range(min(len(tr), len(pr))):
        if tr[j] == pr[j]:
            correct_char += 1
            
    if pr == tr :
        correct += 1 
    
print('Correct characters predicted : %.2f%%' %(correct_char*100/total_char))
print('Correct words predicted      : %.2f%%' %(correct*100/valid_size))

In [ ]:
test = pd.read_csv('/kaggle/input/handwriting-recognition/written_name_test_v2.csv')

plt.figure(figsize=(15, 10))
for i in range(6):
    ax = plt.subplot(2, 3, i+1)
    img_dir = '/kaggle/input/handwriting-recognition/test_v2/test/'+test.loc[i, 'FILENAME']
    image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    plt.imshow(image, cmap='gray')
    
    image = preprocess(image)
    image = image/255.
    pred = model.predict(image.reshape(1, 256, 64, 1))
    decoded = K.get_value(K.ctc_decode(pred, input_length=np.ones(pred.shape[0])*pred.shape[1], 
                                       greedy=True)[0][0])
    plt.title(num_to_label(decoded[0]), fontsize=12)
    plt.axis('off')
    
plt.subplots_adjust(wspace=0.2, hspace=-0.8)